In [6]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np

from dataset import Balls_CF_Detection
from dataset_det import COLORS

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 50, 3, 1)
        self.conv2 = nn.Conv2d(50, 100, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(52900, 64)
        self.fc2 = nn.Linear(64, 9)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x


def train(model, device, train_loader, optimizer, epoch):
    model.train()
        
    batch_idx = 0
    test_loss  = 0
    correct = 0
    total = 0
    for data, target in train_loader:
        #data = data.squeeze()
            
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
            
        output = model(data)
        
        loss = nn.BCEWithLogitsLoss()(output, target)
        loss.backward()
        optimizer.step()
        
        test_loss += loss
        
        output = (torch.sigmoid(output)>0.5).float()
        total += 9 * target.size(0)
        correct += sum(sum(output == target))
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        batch_idx += 1
        
    test_loss /= len(test_loader.dataset) 
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, total,
    100. * correct / total))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    cm = []
    for i in range(0,9):
        cm.append( np.zeros((2, 2), dtype=int) )
    
    with torch.no_grad():
        for data, target in train_loader:
            #data = data.squeeze()
            
            data, target = data.to(device), target.to(device)
            output = model(data)            
            test_loss += nn.BCEWithLogitsLoss()(output, target)  # sum up batch loss
                    
            output = (torch.sigmoid(output)>0.5).float()
            
            # Build the confusion matrix
            #for i in range(0,target.size(0)):
            #    for j in range(0,9):
            #        np.add.at(cm[j], (int(output[i][j]), int(target[i][j])), 1)
                
            total += 9 * target.size(0)
            correct += sum(sum(output == target))
            
        test_loss /= len(test_loader.dataset) 

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, total,
            100. * correct / total))

        #for i in range(0,9):
        #    print(COLORS[i])
        #    print(cm[i])

# Training settings
use_cuda = torch.cuda.is_available()
#use_cuda = False

torch.manual_seed(0)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 100}
test_kwargs = {'batch_size': 100}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


dset = Balls_CF_Detection("data/train/train/")
dval = Balls_CF_Detection("data/train/val/")

dataset1 = dset
dataset2 = dval

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.002)

scheduler = StepLR(optimizer, step_size=1, gamma=0.9216)
#for epoch in range(0, 30):
#    train(model, device, train_loader, optimizer, epoch)
#    test(model, device, test_loader)
#    scheduler.step()


In [4]:
for epoch in range(30, 60):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 30 [0/16800 (0%)]	Loss: 0.332447
Train Epoch: 30 [100/16800 (1%)]	Loss: 0.335809
Train Epoch: 30 [200/16800 (1%)]	Loss: 0.326695
Train Epoch: 30 [300/16800 (2%)]	Loss: 0.329856
Train Epoch: 30 [400/16800 (2%)]	Loss: 0.342445
Train Epoch: 30 [500/16800 (3%)]	Loss: 0.344006
Train Epoch: 30 [600/16800 (4%)]	Loss: 0.340139
Train Epoch: 30 [700/16800 (4%)]	Loss: 0.348875
Train Epoch: 30 [800/16800 (5%)]	Loss: 0.341387
Train Epoch: 30 [900/16800 (5%)]	Loss: 0.337474
Train Epoch: 30 [1000/16800 (6%)]	Loss: 0.336930
Train Epoch: 30 [1100/16800 (7%)]	Loss: 0.353619
Train Epoch: 30 [1200/16800 (7%)]	Loss: 0.329338
Train Epoch: 30 [1300/16800 (8%)]	Loss: 0.347745
Train Epoch: 30 [1400/16800 (8%)]	Loss: 0.325006
Train Epoch: 30 [1500/16800 (9%)]	Loss: 0.323472
Train Epoch: 30 [1600/16800 (10%)]	Loss: 0.346683
Train Epoch: 30 [1700/16800 (10%)]	Loss: 0.317441
Train Epoch: 30 [1800/16800 (11%)]	Loss: 0.333128
Train Epoch: 30 [1900/16800 (11%)]	Loss: 0.346509
Train Epoch: 30 [2000/16800 

Train Epoch: 30 [16400/16800 (98%)]	Loss: 0.328968
Train Epoch: 30 [16500/16800 (98%)]	Loss: 0.347840
Train Epoch: 30 [16600/16800 (99%)]	Loss: 0.339115
Train Epoch: 30 [16700/16800 (99%)]	Loss: 0.344045

Train set: Average loss: 0.0135, Accuracy: 124936/151200 (83%)


Test set: Average loss: 0.0106, Accuracy: 130277/151200 (86%)

Train Epoch: 31 [0/16800 (0%)]	Loss: 0.319217
Train Epoch: 31 [100/16800 (1%)]	Loss: 0.325867
Train Epoch: 31 [200/16800 (1%)]	Loss: 0.364167
Train Epoch: 31 [300/16800 (2%)]	Loss: 0.329817
Train Epoch: 31 [400/16800 (2%)]	Loss: 0.315439
Train Epoch: 31 [500/16800 (3%)]	Loss: 0.331473
Train Epoch: 31 [600/16800 (4%)]	Loss: 0.319503
Train Epoch: 31 [700/16800 (4%)]	Loss: 0.340496
Train Epoch: 31 [800/16800 (5%)]	Loss: 0.337825
Train Epoch: 31 [900/16800 (5%)]	Loss: 0.342671
Train Epoch: 31 [1000/16800 (6%)]	Loss: 0.354829
Train Epoch: 31 [1100/16800 (7%)]	Loss: 0.332389
Train Epoch: 31 [1200/16800 (7%)]	Loss: 0.347245
Train Epoch: 31 [1300/16800 (8%)]	Loss: 0.

Train Epoch: 31 [15700/16800 (93%)]	Loss: 0.346816
Train Epoch: 31 [15800/16800 (94%)]	Loss: 0.323054
Train Epoch: 31 [15900/16800 (95%)]	Loss: 0.336286
Train Epoch: 31 [16000/16800 (95%)]	Loss: 0.331620
Train Epoch: 31 [16100/16800 (96%)]	Loss: 0.325595
Train Epoch: 31 [16200/16800 (96%)]	Loss: 0.357209
Train Epoch: 31 [16300/16800 (97%)]	Loss: 0.317307
Train Epoch: 31 [16400/16800 (98%)]	Loss: 0.333505
Train Epoch: 31 [16500/16800 (98%)]	Loss: 0.314819
Train Epoch: 31 [16600/16800 (99%)]	Loss: 0.328885
Train Epoch: 31 [16700/16800 (99%)]	Loss: 0.321919

Train set: Average loss: 0.0134, Accuracy: 125282/151200 (83%)


Test set: Average loss: 0.0106, Accuracy: 130372/151200 (86%)

Train Epoch: 32 [0/16800 (0%)]	Loss: 0.332887
Train Epoch: 32 [100/16800 (1%)]	Loss: 0.319369
Train Epoch: 32 [200/16800 (1%)]	Loss: 0.336345
Train Epoch: 32 [300/16800 (2%)]	Loss: 0.341839
Train Epoch: 32 [400/16800 (2%)]	Loss: 0.340226
Train Epoch: 32 [500/16800 (3%)]	Loss: 0.325961
Train Epoch: 32 [600/168

Train Epoch: 32 [15000/16800 (89%)]	Loss: 0.351863
Train Epoch: 32 [15100/16800 (90%)]	Loss: 0.337551
Train Epoch: 32 [15200/16800 (90%)]	Loss: 0.330557
Train Epoch: 32 [15300/16800 (91%)]	Loss: 0.340324
Train Epoch: 32 [15400/16800 (92%)]	Loss: 0.350832
Train Epoch: 32 [15500/16800 (92%)]	Loss: 0.329028
Train Epoch: 32 [15600/16800 (93%)]	Loss: 0.320953
Train Epoch: 32 [15700/16800 (93%)]	Loss: 0.331891
Train Epoch: 32 [15800/16800 (94%)]	Loss: 0.308970
Train Epoch: 32 [15900/16800 (95%)]	Loss: 0.342737
Train Epoch: 32 [16000/16800 (95%)]	Loss: 0.321020
Train Epoch: 32 [16100/16800 (96%)]	Loss: 0.335749
Train Epoch: 32 [16200/16800 (96%)]	Loss: 0.318363
Train Epoch: 32 [16300/16800 (97%)]	Loss: 0.330032
Train Epoch: 32 [16400/16800 (98%)]	Loss: 0.331967
Train Epoch: 32 [16500/16800 (98%)]	Loss: 0.320337
Train Epoch: 32 [16600/16800 (99%)]	Loss: 0.343499
Train Epoch: 32 [16700/16800 (99%)]	Loss: 0.334287

Train set: Average loss: 0.0132, Accuracy: 125599/151200 (83%)


Test set: Averag

Train Epoch: 33 [14300/16800 (85%)]	Loss: 0.322205
Train Epoch: 33 [14400/16800 (86%)]	Loss: 0.333143
Train Epoch: 33 [14500/16800 (86%)]	Loss: 0.325357
Train Epoch: 33 [14600/16800 (87%)]	Loss: 0.331003
Train Epoch: 33 [14700/16800 (88%)]	Loss: 0.308630
Train Epoch: 33 [14800/16800 (88%)]	Loss: 0.333530
Train Epoch: 33 [14900/16800 (89%)]	Loss: 0.326754
Train Epoch: 33 [15000/16800 (89%)]	Loss: 0.327052
Train Epoch: 33 [15100/16800 (90%)]	Loss: 0.320545
Train Epoch: 33 [15200/16800 (90%)]	Loss: 0.347918
Train Epoch: 33 [15300/16800 (91%)]	Loss: 0.317753
Train Epoch: 33 [15400/16800 (92%)]	Loss: 0.315903
Train Epoch: 33 [15500/16800 (92%)]	Loss: 0.327138
Train Epoch: 33 [15600/16800 (93%)]	Loss: 0.301293
Train Epoch: 33 [15700/16800 (93%)]	Loss: 0.328040
Train Epoch: 33 [15800/16800 (94%)]	Loss: 0.336440
Train Epoch: 33 [15900/16800 (95%)]	Loss: 0.338924
Train Epoch: 33 [16000/16800 (95%)]	Loss: 0.345214
Train Epoch: 33 [16100/16800 (96%)]	Loss: 0.343990
Train Epoch: 33 [16200/16800 (9

...

Train Epoch: 55 [15000/16800 (89%)]	Loss: 0.298079
Train Epoch: 55 [15100/16800 (90%)]	Loss: 0.308031
Train Epoch: 55 [15200/16800 (90%)]	Loss: 0.311811
Train Epoch: 55 [15300/16800 (91%)]	Loss: 0.301436
Train Epoch: 55 [15400/16800 (92%)]	Loss: 0.336273
Train Epoch: 55 [15500/16800 (92%)]	Loss: 0.330274
Train Epoch: 55 [15600/16800 (93%)]	Loss: 0.284832
Train Epoch: 55 [15700/16800 (93%)]	Loss: 0.337006
Train Epoch: 55 [15800/16800 (94%)]	Loss: 0.292747
Train Epoch: 55 [15900/16800 (95%)]	Loss: 0.316932
Train Epoch: 55 [16000/16800 (95%)]	Loss: 0.314034
Train Epoch: 55 [16100/16800 (96%)]	Loss: 0.305278
Train Epoch: 55 [16200/16800 (96%)]	Loss: 0.314876
Train Epoch: 55 [16300/16800 (97%)]	Loss: 0.339613
Train Epoch: 55 [16400/16800 (98%)]	Loss: 0.309408
Train Epoch: 55 [16500/16800 (98%)]	Loss: 0.322643
Train Epoch: 55 [16600/16800 (99%)]	Loss: 0.302103
Train Epoch: 55 [16700/16800 (99%)]	Loss: 0.329249

Train set: Average loss: 0.0125, Accuracy: 127353/151200 (84%)


Test set: Averag

Train Epoch: 56 [14300/16800 (85%)]	Loss: 0.319821
Train Epoch: 56 [14400/16800 (86%)]	Loss: 0.354836
Train Epoch: 56 [14500/16800 (86%)]	Loss: 0.312177
Train Epoch: 56 [14600/16800 (87%)]	Loss: 0.310961
Train Epoch: 56 [14700/16800 (88%)]	Loss: 0.299003
Train Epoch: 56 [14800/16800 (88%)]	Loss: 0.335843
Train Epoch: 56 [14900/16800 (89%)]	Loss: 0.296739
Train Epoch: 56 [15000/16800 (89%)]	Loss: 0.301419
Train Epoch: 56 [15100/16800 (90%)]	Loss: 0.299697
Train Epoch: 56 [15200/16800 (90%)]	Loss: 0.300812
Train Epoch: 56 [15300/16800 (91%)]	Loss: 0.326057
Train Epoch: 56 [15400/16800 (92%)]	Loss: 0.295992
Train Epoch: 56 [15500/16800 (92%)]	Loss: 0.312846
Train Epoch: 56 [15600/16800 (93%)]	Loss: 0.340869
Train Epoch: 56 [15700/16800 (93%)]	Loss: 0.318906
Train Epoch: 56 [15800/16800 (94%)]	Loss: 0.282216
Train Epoch: 56 [15900/16800 (95%)]	Loss: 0.306291
Train Epoch: 56 [16000/16800 (95%)]	Loss: 0.315734
Train Epoch: 56 [16100/16800 (96%)]	Loss: 0.297286
Train Epoch: 56 [16200/16800 (9

Train Epoch: 57 [13600/16800 (81%)]	Loss: 0.323171
Train Epoch: 57 [13700/16800 (82%)]	Loss: 0.320797
Train Epoch: 57 [13800/16800 (82%)]	Loss: 0.333971
Train Epoch: 57 [13900/16800 (83%)]	Loss: 0.292017
Train Epoch: 57 [14000/16800 (83%)]	Loss: 0.295237
Train Epoch: 57 [14100/16800 (84%)]	Loss: 0.289985
Train Epoch: 57 [14200/16800 (85%)]	Loss: 0.317621
Train Epoch: 57 [14300/16800 (85%)]	Loss: 0.309020
Train Epoch: 57 [14400/16800 (86%)]	Loss: 0.322311
Train Epoch: 57 [14500/16800 (86%)]	Loss: 0.317447
Train Epoch: 57 [14600/16800 (87%)]	Loss: 0.298729
Train Epoch: 57 [14700/16800 (88%)]	Loss: 0.317469
Train Epoch: 57 [14800/16800 (88%)]	Loss: 0.307539
Train Epoch: 57 [14900/16800 (89%)]	Loss: 0.307347
Train Epoch: 57 [15000/16800 (89%)]	Loss: 0.317875
Train Epoch: 57 [15100/16800 (90%)]	Loss: 0.312214
Train Epoch: 57 [15200/16800 (90%)]	Loss: 0.309168
Train Epoch: 57 [15300/16800 (91%)]	Loss: 0.315400
Train Epoch: 57 [15400/16800 (92%)]	Loss: 0.307501
Train Epoch: 57 [15500/16800 (9

Train Epoch: 58 [12900/16800 (77%)]	Loss: 0.301838
Train Epoch: 58 [13000/16800 (77%)]	Loss: 0.298705
Train Epoch: 58 [13100/16800 (78%)]	Loss: 0.289938
Train Epoch: 58 [13200/16800 (79%)]	Loss: 0.302123
Train Epoch: 58 [13300/16800 (79%)]	Loss: 0.337351
Train Epoch: 58 [13400/16800 (80%)]	Loss: 0.305823
Train Epoch: 58 [13500/16800 (80%)]	Loss: 0.316641
Train Epoch: 58 [13600/16800 (81%)]	Loss: 0.309822
Train Epoch: 58 [13700/16800 (82%)]	Loss: 0.354348
Train Epoch: 58 [13800/16800 (82%)]	Loss: 0.302295
Train Epoch: 58 [13900/16800 (83%)]	Loss: 0.326347
Train Epoch: 58 [14000/16800 (83%)]	Loss: 0.316718
Train Epoch: 58 [14100/16800 (84%)]	Loss: 0.291905
Train Epoch: 58 [14200/16800 (85%)]	Loss: 0.290185
Train Epoch: 58 [14300/16800 (85%)]	Loss: 0.327076
Train Epoch: 58 [14400/16800 (86%)]	Loss: 0.289884
Train Epoch: 58 [14500/16800 (86%)]	Loss: 0.314071
Train Epoch: 58 [14600/16800 (87%)]	Loss: 0.320419
Train Epoch: 58 [14700/16800 (88%)]	Loss: 0.304993
Train Epoch: 58 [14800/16800 (8

Train Epoch: 59 [12300/16800 (73%)]	Loss: 0.321379
Train Epoch: 59 [12400/16800 (74%)]	Loss: 0.288133
Train Epoch: 59 [12500/16800 (74%)]	Loss: 0.320381
Train Epoch: 59 [12600/16800 (75%)]	Loss: 0.305329
Train Epoch: 59 [12700/16800 (76%)]	Loss: 0.322307
Train Epoch: 59 [12800/16800 (76%)]	Loss: 0.314254
Train Epoch: 59 [12900/16800 (77%)]	Loss: 0.290652
Train Epoch: 59 [13000/16800 (77%)]	Loss: 0.308601
Train Epoch: 59 [13100/16800 (78%)]	Loss: 0.278771
Train Epoch: 59 [13200/16800 (79%)]	Loss: 0.323787
Train Epoch: 59 [13300/16800 (79%)]	Loss: 0.290137
Train Epoch: 59 [13400/16800 (80%)]	Loss: 0.297514
Train Epoch: 59 [13500/16800 (80%)]	Loss: 0.313742
Train Epoch: 59 [13600/16800 (81%)]	Loss: 0.305667
Train Epoch: 59 [13700/16800 (82%)]	Loss: 0.310738
Train Epoch: 59 [13800/16800 (82%)]	Loss: 0.295598
Train Epoch: 59 [13900/16800 (83%)]	Loss: 0.290488
Train Epoch: 59 [14000/16800 (83%)]	Loss: 0.297606
Train Epoch: 59 [14100/16800 (84%)]	Loss: 0.300353
Train Epoch: 59 [14200/16800 (8

In [11]:
#torch.save(model, "mnist_99_percent_accuracy")
model = torch.load("mnist_99_percent_accuracy")

In [12]:
model.to(device)
test(model, device, test_loader)


Test set: Average loss: 0.0013, Accuracy: 150373/151200 (99%)

